In [ ]:
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.utils.rnn as rnn_utils
import optuna 
from sklearn.metrics import confusion_matrix, classification_report 


In [ ]:
dir_path = "../DataSet" 
actions = np.array(os.listdir(dir_path))

## formatting the data

In [ ]:
#Pre processing data and creating labels 

label_map = {label: num for num, label in enumerate(actions)}

In [ ]:
labels = []

for action in actions: 
    file_list = os.listdir(dir_path+"/"+action)
    for video in range(len(file_list)): 
        labels.append(label_map[action])


In [ ]:
def extract_frames_and_resize(video_path, frame_size=(244,244)): 
    frames = []
    lengths = []
    cap = cv2.VideoCapture(video_path)
    while True: 
        ret,frame = cap.read()
        if not ret: 
            break 
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
        lengths.append(len(frame))
 
    cap.release()
    return frames, lengths


In [ ]:
data = []
for action in actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
       
    for file in file_list:
        file_path = os.path.join(dir_path, action, file)
        frames, lengths = extract_frames_and_resize(file_path)

        data.append(frames)
    

In [ ]:
data = np.array(data, dtype=list)
labels = np.array(labels)

In [ ]:

data = pad_sequences(data, padding='post')
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.1, random_state=42, stratify=labels )


In [ ]:


device = torch.device("cpu")
lengths_tensor = torch.tensor(lengths)

train_features = torch.tensor(train_data).float().to(device)
train_labels = torch.tensor(train_labels).long().to(device)

test_features = torch.tensor(test_data).float().to(device)
test_labels = torch.tensor(test_labels).long().to(device)

threshold = 0 

In [ ]:


class Model(nn.Module):
    def __init__(self, num_classes, num_hidden1, num_hidden2):
        super(Model, self).__init__()
        self.conv1 = nn.Conv3d(72, num_hidden1, kernel_size=(3, 3, 3), stride=1, padding=0)
        self.pool1 = nn.AdaptiveMaxPool3d((None,112,112))
        self.conv2 = nn.Conv3d(num_hidden1, num_hidden2, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.pool2 = nn.AdaptiveMaxPool3d((None, 56,56))
        self.flatten = nn.Flatten()
        
        num_features = num_hidden2*56*56
        self.fc1 = nn.Linear(num_features, 128) 
        self.fc2 = nn.Linear(128, num_classes)
        self.mask = None 
    
    def apply_mask(self, x, mask): 
        return x * mask
    
    def forward(self, x, mask=None):
        if mask is not None: 
            x= self.apply_mask(x,mask)
        x = x.permute(0, 1, 4, 2, 3)  
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return nn.functional.softmax(x, dim=1)






In [ ]:
model = Model(actions.shape[0],48,64).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
train_dataset = TensorDataset(train_features, train_labels)
train_loader = DataLoader(train_dataset, batch_size=82, shuffle=True)  

test_dataset = TensorDataset(test_features, test_labels)
test_loader = DataLoader(test_dataset, batch_size=82, shuffle=False)


## Hyper parameter tuning


In [ ]:
accumulation_steps = 4 
num_epochs = 500
def train_and_evaluate_model(trial, train_loader, test_loader): 
    learning_rate = trial.suggest_categorical("learning_rate", [0.001, 0.01])
    num_hidden1 = trial.suggest_int("num_hidden1",16,64, step=16 )
    num_hidden2 = trial.suggest_int("num_hidden2", 16, 64, step=16)
    
    model = Model(actions.shape[0], num_hidden1, num_hidden2).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr= learning_rate)
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for i, (features, labels) in enumerate(train_loader):
            features = features.to(device)
            labels = labels.to(device)
            
            mask_tensor = (features > threshold).float().to(device)
            outputs = model(features,mask_tensor)
            loss = criterion(outputs, train_labels)
            loss = loss / accumulation_steps
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            running_loss += loss.item() * accumulation_steps
             


            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted ==labels).sum().item()
        epoch_loss = running_loss / (i + 1)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
    correct_test = 0
    total_test = 0

    # Set the model to evaluation mode
    model.eval()

    # Calculate accuracy on the training set
    with torch.no_grad():
        for features, labels in test_loader:
            features = features.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(features)

            # Get the predicted class labels
            _, predicted = torch.max(outputs, 1)

            # Get the ground truth class labels
            true_labels = labels

            # Update the count of total and correct predictions
            total_test += true_labels.size(0)
            correct_test += (predicted == true_labels).sum().item()

    # Calculate the accuracy
    accuracy = 100 * correct_test / total_test

    print(f'Validation accuracy: {accuracy}%')
    
    return accuracy 
    


In [ ]:

study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: train_and_evaluate_model(trial, train_loader, test_loader), n_trials=10)

print("Best trial:")
print("  Params: ", study.best_trial.params)
print("  Validation accuracy: ", study.best_trial.value)

## Training the model

In [ ]:
accumulation_steps = 4
num_epochs = 500

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)
        
        mask_tensor = (features > threshold).float().to(device)
        outputs = model(features,mask_tensor)
        loss = criterion(outputs, labels)
        loss = loss / accumulation_steps
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item() * accumulation_steps

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / (i + 1)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


In [ ]:
correct = 0
total = 0

# Set the model to evaluation mode
model.eval()

# Calculate accuracy on the training set
with torch.no_grad():
    for features, labels in test_loader:
        features = features.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(features)

        # Get the predicted class labels
        _, predicted = torch.max(outputs, 1)

        # Get the ground truth class labels
        true_labels = labels

        # Update the count of total and correct predictions
        total += true_labels.size(0)
        correct += (predicted == true_labels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total

print(f'Training accuracy: {accuracy}%')


In [ ]:
print(predicted)
print(true_labels)

In [ ]:

report = classification_report(true_labels, predicted)
cm = confusion_matrix(true_labels, predicted)

print(cm)
print(report)

In [ ]:


# The labels for your classes

class_names = ['Blue', 'Family', 'Happy', 'Man']

plt.figure(figsize=(10,7))

sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', xticklabels=class_names, yticklabels=class_names)


plt.title('Confusion Matrix')

plt.ylabel('True label')

plt.xlabel('Predicted label')


plt.show()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print("Active device:", device)